In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
df = pd.read_csv('mro_daily_clean.csv')

### group to weekly level

In [4]:
df.columns

Index(['yr_nbr', 'mth_nbr', 'week_nbr', 'week_day', 'hard_braking', 'mild_hb',
       'hard_braking2', 'harsh_hb', 'very_harsh_hb', 'est_hh_incm_prmr_cd',
       'purchaser_age_at_tm_of_purch', 'input_indiv_gndr_prmr_cd',
       'gmqualty_model', 'umf_xref_finc_gbl_trim', 'engn_size',
       'hard_acceleration', 'mild_ha', 'harsh_ha', 'very_harsh_ha',
       'vin_nbr_id', 'speeding_sum', 'speeding2_sum', 'day_mileage', 'id',
       'day_indicator', 'service_days', 'battery_dummy', 'brake_dummy',
       'tire_dummy', 'lof_dummy', 'wiper_dummy', 'filter_dummy', 'others',
       'mro', 'record_days', 'latitude1', 'longitude1', 'purchase_lat1',
       'purchase_lng1', 'purchase_yr_nbr', 'purchase_mth_nbr', 'tavg',
       'random_avg_traffic'],
      dtype='object')

In [5]:
df[['battery_dummy', 'brake_dummy','tire_dummy', 'lof_dummy', 'wiper_dummy', 'filter_dummy', 'others','mro']].sum() / df['mro'].sum()

battery_dummy    0.003889
brake_dummy      0.001638
tire_dummy       0.018202
lof_dummy        0.620093
wiper_dummy      0.005081
filter_dummy     0.019343
others           0.357591
mro              1.000000
dtype: float64

In [6]:
df['mro_new'] = df['tire_dummy'] + df['lof_dummy'] + df['wiper_dummy'] + df['others']

In [ ]:
df = df[
    [
        "id",
        "yr_nbr",
        "mth_nbr",
        "week_nbr",
        "est_hh_incm_prmr_cd",
        "purchaser_age_at_tm_of_purch",
        "input_indiv_gndr_prmr_cd",
        "gmqualty_model",
        "umf_xref_finc_gbl_trim",
        "engn_size",
        "hard_braking",
        "hard_acceleration",
        "speeding_sum",
        "day_mileage",
        "mro_new",
        "record_days",
        "purchase_yr_nbr",
        "purchase_mth_nbr",
        "tavg",
        "random_avg_traffic",
    ]
]

In [ ]:
df["purchase_time"] = (
    df["purchase_yr_nbr"].astype(int).astype(str)
    + "-"
    + df["purchase_mth_nbr"].astype(int).astype(str)
)
df = df.drop(["purchase_yr_nbr", "purchase_mth_nbr"], axis=1)

In [ ]:
#robustness check: divide into repair/maintenance
df['maintenance'] = np.where((df['mro'] == 1) & (df['service_days'] <= 3), 1, 0)
df['repair'] = np.where((df['mro'] == 1) & (df['service_days'] > 3), 1, 0)

In [ ]:
#robustness check: traffic density
df['traffic_count'] = df.groupby(['yr_nbr', 'mth_nbr', 'week_nbr', 'week_day', 'latitude1', 'longitude1'])['yr_nbr'].transform('size')

In [ ]:
#robustness check: another proxy for driving behavior features
df['hard_braking_sum'] = df['mild_hb'] + df['harsh_hb'] + df['very_harsh_hb']
df['hard_acceleration_sum'] = df['mild_ha'] + df['harsh_ha'] + df['very_harsh_ha']

In [9]:
weekly_level = df.groupby(['id','yr_nbr','week_nbr']).agg({
    'mth_nbr':'first',
    'mro_new':'sum',
    'hard_braking':'sum',
    'hard_acceleration':'sum',
    'speeding_sum':'sum',
    'day_mileage':'sum',
    'est_hh_incm_prmr_cd':'first',
    'purchaser_age_at_tm_of_purch':'first',
    'input_indiv_gndr_prmr_cd':'first',
    'gmqualty_model':'first',
    'umf_xref_finc_gbl_trim':'first',
    'engn_size':'first',
    'purchase_time':'first',
    'tavg':'mean',
    'record_days':'first',
    'random_avg_traffic':'mean'})
weekly_level_new = weekly_level.reset_index()
#grouped = grouped.drop(['yr_nbr','mth_nbr'], axis=1)

In [10]:
weekly_level_new.shape

(3972103, 19)

In [11]:
weekly_level_new.head()

,id,yr_nbr,week_nbr,mth_nbr,mro_new,hard_braking,hard_acceleration,speeding_sum,day_mileage,est_hh_incm_prmr_cd,purchaser_age_at_tm_of_purch,input_indiv_gndr_prmr_cd,gmqualty_model,umf_xref_finc_gbl_trim,engn_size,purchase_time,tavg,record_days,random_avg_traffic
0,w4HClcKnwrzCv8KgwrjDi8Klwr3Cm8KVwqfCrsKowprClg==,2018,1,12,0.0,1,1,0.0,6.203125,6.0,54.0,F,Regal,BASE_TRIM,2.0,2018-1,5.048431,596,12886.225115
1,w4HClcKnwrzCv8KgwrjDi8Klwr3Cm8KVwqfCrsKowprClg==,2018,3,1,0.0,1,0,4.0,32.968750,6.0,54.0,F,Regal,BASE_TRIM,2.0,2018-1,15.092748,596,14554.620499
2,w4HClcKnwrzCv8KgwrjDi8Klwr3Cm8KVwqfCrsKowprClg==,2018,4,1,0.0,39,5,21.0,319.812500,6.0,54.0,F,Regal,BASE_TRIM,2.0,2018-1,15.273847,596,14559.007102
3,w4HClcKnwrzCv8KgwrjDi8Klwr3Cm8KVwqfCrsKowprClg==,2018,5,1,0.0,60,12,33.0,541.687500,6.0,54.0,F,Regal,BASE_TRIM,2.0,2018-1,15.005342,596,14433.908044
4,w4HClcKnwrzCv8KgwrjDi8Klwr3Cm8KVwqfCrsKowprClg==,2018,6,2,0.0,50,10,46.0,352.406250,6.0,54.0,F,Regal,BASE_TRIM,2.0,2018-1,14.859090,596,14390.440682


In [12]:
weekly_level_new['mro_new'][weekly_level_new['mro_new'] >= 1] = 1

C:\Users\Jieyi Zhang\AppData\Local\Temp\ipykernel_13272\1444146333.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weekly_level_new['mro_new'][weekly_level_new['mro_new'] >= 1] = 1


In [ ]:
weekly_level_new['mro'][weekly_level_new['mro'] >= 1] = 1
weekly_level_new['repair'][weekly_level_new['repair'] >= 1] = 1
weekly_level_new['maintenance'][weekly_level_new['maintenance'] >= 1] = 1

### Clean weekly level data for prediction

#### Step1. Select vehicles with records >= 120 days.

In [13]:
weekly_level_filter = weekly_level_new[weekly_level_new['record_days'] >= 120]
weekly_level_filter.shape

(3434762, 19)

In [14]:
len(weekly_level_filter['id'].unique())

58839

#### Step2. Remove Outliers.

In [15]:
low = weekly_level_filter['random_avg_traffic'].quantile(0.025)
high = weekly_level_filter['random_avg_traffic'].quantile(0.975)

In [16]:
weekly_level_filter.loc[weekly_level_filter['random_avg_traffic'] <= low, 'random_avg_traffic'] = low
weekly_level_filter.loc[weekly_level_filter['random_avg_traffic'] >= high, 'random_avg_traffic'] = high

In [17]:
scaler = MinMaxScaler()
weekly_level_filter['random_avg_traffic'] = scaler.fit_transform(weekly_level_filter[['random_avg_traffic']])

C:\Users\Jieyi Zhang\AppData\Local\Temp\ipykernel_13272\1365610199.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weekly_level_filter['random_avg_traffic'] = scaler.fit_transform(weekly_level_filter[['random_avg_traffic']])


In [18]:
weekly_level_filter.columns

Index(['id', 'yr_nbr', 'week_nbr', 'mth_nbr', 'mro_new', 'hard_braking',
       'hard_acceleration', 'speeding_sum', 'day_mileage',
       'est_hh_incm_prmr_cd', 'purchaser_age_at_tm_of_purch',
       'input_indiv_gndr_prmr_cd', 'gmqualty_model', 'umf_xref_finc_gbl_trim',
       'engn_size', 'purchase_time', 'tavg', 'record_days',
       'random_avg_traffic'],
      dtype='object')

In [39]:
id_info = weekly_level_filter.groupby('id').agg({
    'mro':'sum',
    'record_weeks':'first',
    'hard_braking':'mean',
    'hard_acceleration':'mean',
    'speeding_sum':'mean',
    'day_mileage': 'mean',
    'est_hh_incm_prmr_cd':'first',
    'purchaser_age_at_tm_of_purch':'first',
    'input_indiv_gndr_prmr_cd':'first',
    'gmqualty_model':'first',
    'umf_xref_finc_gbl_trim':'first',
    'engn_size':'first',
    'purchase_time':'first',
    'tavg':'mean',
    'random_avg_traffic':'mean'
})
id_info['mro_freq'] = id_info['mro'] / id_info['record_weeks']

In [19]:
df = weekly_level_filter.copy()
df['input_indiv_gndr_prmr_cd'] = df['input_indiv_gndr_prmr_cd'].replace({'F': 0, 'M': 1})
df['input_indiv_gndr_prmr_cd'] = df['input_indiv_gndr_prmr_cd'].astype('int')
#df['Hard Braking per mile'] = df['hard_braking_sum'] / df['day_mileage']
#df['Hard Acceleration per mile'] = df['hard_acceleration_sum'] / df['day_mileage']
#df['Speeding per mile'] = df['speeding2_sum'] / df['day_mileage']
#df = df.drop(['hard_braking_sum', 'hard_acceleration_sum', 'speeding2_sum', 'day_mileage'], axis=1)
df = df.fillna(0)

In [61]:
low = df['traffic_count'].quantile(0.025)
high = df['traffic_count'].quantile(0.975)
df.loc[df['traffic_count'] <= low, 'traffic_count'] = low
df.loc[df['traffic_count'] >= high, 'traffic_count'] = high
scaler = MinMaxScaler()
df['traffic_2'] = scaler.fit_transform(df[['traffic_count']])
weekly_level_filter = df.drop('traffic_count', axis=1)

In [23]:
weekly_level_filter = df.copy()
columns_to_filter = ['hard_braking','hard_acceleration', 'speeding_sum', 'day_mileage']

percentile_5 = weekly_level_filter[columns_to_filter].quantile(0.025)
percentile_95 = weekly_level_filter[columns_to_filter].quantile(0.975)

filtered_data = weekly_level_filter[columns_to_filter].apply(
    lambda x: x[(x >= percentile_5[x.name]) & (x <= percentile_95[x.name])]
)

filtered_indices = filtered_data.dropna().index
weekly_level_filter_1 = weekly_level_filter.loc[filtered_indices]

In [24]:
weekly_level_filter_1.shape

(3071095, 19)

In [25]:
weekly_level_filter_1.head()

,id,yr_nbr,week_nbr,mth_nbr,mro_new,hard_braking,hard_acceleration,speeding_sum,day_mileage,est_hh_incm_prmr_cd,purchaser_age_at_tm_of_purch,input_indiv_gndr_prmr_cd,gmqualty_model,umf_xref_finc_gbl_trim,engn_size,purchase_time,tavg,record_days,random_avg_traffic
1,w4HClcKnwrzCv8KgwrjDi8Klwr3Cm8KVwqfCrsKowprClg==,2018,3,1,0.0,1,0,4.0,32.968750,6.0,54.0,0,Regal,BASE_TRIM,2.0,2018-1,15.092748,596,0.246767
2,w4HClcKnwrzCv8KgwrjDi8Klwr3Cm8KVwqfCrsKowprClg==,2018,4,1,0.0,39,5,21.0,319.812500,6.0,54.0,0,Regal,BASE_TRIM,2.0,2018-1,15.273847,596,0.246881
3,w4HClcKnwrzCv8KgwrjDi8Klwr3Cm8KVwqfCrsKowprClg==,2018,5,1,0.0,60,12,33.0,541.687500,6.0,54.0,0,Regal,BASE_TRIM,2.0,2018-1,15.005342,596,0.243615
5,w4HClcKnwrzCv8KgwrjDi8Klwr3Cm8KVwqfCrsKowprClg==,2018,7,2,0.0,38,5,22.0,267.828125,6.0,54.0,0,Regal,BASE_TRIM,2.0,2018-1,15.075015,596,0.242279
6,w4HClcKnwrzCv8KgwrjDi8Klwr3Cm8KVwqfCrsKowprClg==,2018,8,2,0.0,30,0,12.0,248.062500,6.0,54.0,0,Regal,BASE_TRIM,2.0,2018-1,15.307328,596,0.241633


In [28]:
weekly_level_filter_1['record_weeks'] = weekly_level_filter.groupby('id')['id'].transform('size')
filtered_df = weekly_level_filter_1[weekly_level_filter_1['record_weeks'] >= 16]
len(filtered_df['id'].unique())

58839

In [29]:
filtered_df['record_weeks'].min(), filtered_df['record_weeks'].mean(), filtered_df['record_weeks'].max()

(19, 66.16711107927303, 104)

In [30]:
weekly_level_filter_2 = filtered_df.drop('record_days',axis=1)
weekly_level_filter_2 = filtered_df.drop('record_weeks',axis=1)

In [31]:
weekly_level_filter_2.to_csv('weekly_filter_new_2.csv')